In [1]:
import tensorflow as tf

In [ ]:
# class CustomMSE(keras.losses.Loss):
#     def __init__(self, regularization_factor=0.1, name="custom_mse"):
#         super().__init__(name=name)
#         self.regularization_factor = regularization_factor

#     def call(self, y_true, y_pred):
#         mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
#         reg = tf.math.reduce_mean(tf.square(0.5 - y_pred))
#         return mse + reg * self.regularization_factor


In [ ]:
# def cat_cross(y_true, y_pred, weights, epsilon = 1e-9):
#     y_true = tf.convert_to_tensor(y_true, tf.float32)
#     y_pred = tf.convert_to_tensor(y_pred, tf.float32)
#     weights = tf.convert_to_tensor(weights, tf.float32)
#     return -tf.reduce_mean(tf.reduce_sum(weights* y_true* tf.math.log(y_pred + epsilon), axis = [-1]))

In [ ]:
class CustomWeightedCategoricalCrossentropy(tf.keras.losses.Loss):
    def __init__(self, class_weights, epsilon = 1e-9, name = 'wce'):
        super().__init__(name=name)
        self.class_weights = tf.convert_to_tensor(class_weights, tf.float32)
        self.epsilon = epsilon
    def call(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        return -tf.reduce_mean(tf.reduce_sum(self.class_weights* y_true* tf.math.log(y_pred + self.epsilon), axis = [-1]))


In [ ]:
# TEST_CASE
from unet import get_unet_128
import numpy as np
model = get_unet_128(num_classes = 3)
x = np.random.uniform(0,1, (10,128,128,3))
target = np.random.randint(0,2, (10,128,128,3))
prediction = model(x, training = True)


l = CustomWeightedCategoricalCrossentropy([1,1,1])
print(l(target, prediction))

l = tf.keras.losses.CategoricalCrossentropy()
print(l(target, prediction))



In [24]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, tf.float32)
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [33]:
class CustomDiceLoss(tf.keras.losses.Loss):
    def __init__(self, smooth = 1.0 ,name = 'dice_loss'):
        super().__init__(name = name)
        self.smooth = smooth

    def compute_dice_coef(self, y_true, y_pred):
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
        union = tf.reduce_sum(y_true, [1,2,3]) + tf.reduce_sum(y_pred, [1,2,3])
        score = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return score

    def call(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        score  = self.compute_dice_coef(y_true, y_pred)
        return 1 - score
        


In [34]:
# TEST_CASE
from unet import get_unet_128
import numpy as np
import tensorflow.keras.backend as K
model = get_unet_128(num_classes = 3)
x = np.random.uniform(0,1, (10,128,128,3))
target = np.random.randint(0,2, (10,128,128,3))
prediction = model(x, training = True)


l = CustomDiceLoss()
print(l(target, prediction))
print()
print(dice_loss(target, prediction))

# # l = tf.keras.losses.CategoricalCrossentropy()
# print(dice_loss(target.astype(np.float32)[...,0][..., np.newaxis], prediction[...,0][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,1][..., np.newaxis], prediction[...,1][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,2][..., np.newaxis], prediction[...,2][..., np.newaxis]))



tf.Tensor(0.60010135, shape=(), dtype=float32)

tf.Tensor(0.600111, shape=(), dtype=float32)


In [ ]:
class CustomDiceCoefficient(tf.keras.metrics.Metric):
    def __init__(self, n_classes, threshold = 0.5, smooth= 1.0, average='macro', name = 'dice_coef', **kwargs):
        super().__init__(name = name, **kwargs)
        self.n_classes = n_classes
        self.smooth = smooth
        self.threshold = threshold
        self.average = average
            
        self.sum = self.add_weight(name = 'sum', initializer = 'zeros', shape = (self.n_classes,))
        self.total = self.add_weight(name = 'total', initializer = 'zeros')
        self.h_iou = self.add_weight(name = 'dice_coef', initializer = 'zeros')
    
    def update_state(self, y_true, y_pred, sample_weight = None): 
        correct_values = tf.reduce_sum(tf.cast(tf.argmax(y_pred, axis = 1) == tf.argmax(y_true, axis = 1), "float32"))
        y_true = tf.cast(tf.cast(y_true, tf.int32), tf.float32)
        if self.threshold is not None:
            y_pred = tf.cast(tf.cast(y_pred > self.threshold, tf.int32), tf.float32)
        else:
            y_pred = tf.cast(tf.one_hot( tf.argmax(y_pred, axis = -1), self.n_classes  ), tf.float32)
        intersection = tf.reduce_sum(tf.abs(y_true * y_pred), axis=[1,2])
        union = tf.reduce_sum(y_true, [1,2]) + tf.reduce_sum(y_pred, [1,2]) - intersection
        iou = (intersection + self.smooth) / (union + self.smooth)
        
        self.sum.assign_add(tf.reduce_sum(iou, axis = 0))
        self.total.assign_add(tf.cast(tf.shape(y_true)[0], "float32"))
            
        if self.average == 'macro':
            self.h_iou.assign(tf.math.divide_no_nan(tf.reduce_sum(self.weight), tf.reduce_sum(tf.math.divide_no_nan( self.weight, tf.math.divide_no_nan(self.sum, self.total) ))))
        elif self.average == 'micro':
            self.h_iou.assign( tf.reduce_mean(self.weight * tf.math.divide_no_nan(self.sum, self.total)) )
        else:
            raise NameError
        
    def result(self):
        return self.h_iou
    
    def reset_states(self):
        self.sum.assign(self.sum * 0.0)
        self.total.assign(0.0)
        self.h_iou.assign(0.0)

In [9]:
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = (intersection + smooth) / (union + smooth)
  return iou

In [15]:
class CustomIOU(tf.keras.metrics.Metric):
    def __init__(self, n_classes, threshold = None, smooth= 1.0, average='macro', weight = None, name = 'weighted_iou', **kwargs):
        super().__init__(name = name, **kwargs)
        self.n_classes = n_classes
        self.smooth = smooth
        self.threshold = threshold
        self.average = average
        if weight is None:
            self.weight = tf.ones((n_classes,), tf.float32)
        else:
            self.weight = tf.convert_to_tensor(weight, tf.float32)
            
        self.sum = self.add_weight(name = 'sum', initializer = 'zeros', shape = (self.n_classes,))
        self.total = self.add_weight(name = 'total', initializer = 'zeros')
        self.h_iou = self.add_weight(name = 'h_iou', initializer = 'zeros')
    
    def update_state(self, y_true, y_pred, sample_weight = None): 
        y_true = tf.cast(tf.cast(y_true, tf.int32), tf.float32)
        if self.threshold is not None:
            y_pred = tf.cast(tf.cast(y_pred > self.threshold, tf.int32), tf.float32)
        else:
            y_pred = tf.cast(tf.one_hot( tf.argmax(y_pred, axis = -1), self.n_classes  ), tf.float32)
        intersection = tf.reduce_sum(tf.abs(y_true * y_pred), axis=[1,2])
        union = tf.reduce_sum(y_true, [1,2]) + tf.reduce_sum(y_pred, [1,2]) - intersection
        iou = (intersection + self.smooth) / (union + self.smooth)
        
        self.sum.assign_add(tf.reduce_sum(iou, axis = 0))
        self.total.assign_add(tf.cast(tf.shape(y_true)[0], "float32"))
            
        if self.average == 'macro':
            self.h_iou.assign(tf.math.divide_no_nan(tf.reduce_sum(self.weight), tf.reduce_sum(tf.math.divide_no_nan( self.weight, tf.math.divide_no_nan(self.sum, self.total) ))))
        elif self.average == 'micro':
            self.h_iou.assign( tf.reduce_mean(self.weight * tf.math.divide_no_nan(self.sum, self.total)) )
        else:
            raise NameError
        
    def result(self):
        return self.h_iou
    
    def reset_states(self):
        self.sum.assign(self.sum * 0.0)
        self.total.assign(0.0)
        self.h_iou.assign(0.0)

In [16]:
from unet import get_unet_128
import numpy as np
import tensorflow.keras.backend as K
model = get_unet_128(num_classes = 3)
x = np.random.uniform(0,1, (10,128,128,3))
target = np.random.randint(0,2, (10,128,128,3))
prediction = model(x, training = False)
prediction_classes = tf.cast(tf.one_hot( tf.argmax(prediction, axis = -1), 3  ), tf.float32)

m = CustomWeightedIOU(3)
m.update_state(target, prediction)
m.result()

tf.Tensor(
[0.49306408 0.49815702 0.5038641  0.49650478 0.49640596 0.49932417
 0.49223307 0.49229163 0.4938605  0.49797398], shape=(10,), dtype=float32)
tf.Tensor(
[0.00589825 0.00748396 0.00671221 0.00580411 0.0073432  0.00609088
 0.00658778 0.00871226 0.00701585 0.00530376], shape=(10,), dtype=float32)
tf.Tensor(
[0.00602845 0.00528465 0.00610559 0.00522987 0.00629998 0.00684011
 0.0066361  0.00539018 0.00561661 0.00571985], shape=(10,), dtype=float32)


In [46]:
def tversky_index(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.7
    smooth = 1.0
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (
                1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, tf.float32)
    return 1 - tversky_index(y_true, y_pred)

# def focal_tversky(y_true, y_pred):
#     pt_1 = tversky_index(y_true, y_pred)
#     gamma = 0.75
#     return K.pow((1 - pt_1), gamma)

# def log_cosh_dice_loss(y_true, y_pred):b
#     x = dice_loss(y_true, y_pred)
#     return tf.math.log((tf.exp(x) + tf.exp(-x)) / 2.0)

In [47]:
class CustomTverskyLoss(tf.keras.losses.Loss):
    def __init__(self, alpha = 0.7, smooth = 1.0 ,name = 'tversky_loss'):
        super().__init__(name = name)
        self.smooth = smooth
        self.alpha = alpha

    def compute_tversky_index(self, y_true, y_pred):
        true_pos = tf.reduce_sum(y_true * y_pred)
        false_neg = tf.reduce_sum(y_true * (1 - y_pred))
        false_pos = tf.reduce_sum((1 - y_true) * y_pred)

        ti = (true_pos + self.smooth) / (true_pos + self.alpha * false_neg + (1 - self.alpha) * false_pos + self.smooth)
        return ti

    def call(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        score  = self.compute_tversky_index(y_true, y_pred)
        return 1 - score
        


In [48]:
# TEST_CASE
from unet import get_unet_128
import numpy as np
import tensorflow.keras.backend as K
model = get_unet_128(num_classes = 3)
x = np.random.uniform(0,1, (10,128,128,3))
target = np.random.randint(0,2, (10,128,128,3))
prediction = model(x, training = True)


l = CustomTverskyLoss()
print(l(target, prediction))
print()
print(tversky_loss(target, prediction))

# # l = tf.keras.losses.CategoricalCrossentropy()
# print(dice_loss(target.astype(np.float32)[...,0][..., np.newaxis], prediction[...,0][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,1][..., np.newaxis], prediction[...,1][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,2][..., np.newaxis], prediction[...,2][..., np.newaxis]))



tf.Tensor(0.6297133, shape=(), dtype=float32)

tf.Tensor(0.6297133, shape=(), dtype=float32)


In [52]:
def focal_tversky(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, tf.float32)
    pt_1 = tversky_index(y_true, y_pred)
    gamma = 0.75
    return K.pow((1 - pt_1), gamma)

In [53]:
class CustomFocalTverskyLoss(tf.keras.losses.Loss):
    def __init__(self, alpha = 0.7, gamma = 0.75, smooth = 1.0 ,name = 'focal_tversky'):
        super().__init__(name = name)
        self.smooth = smooth
        self.gamma = gamma
        self.alpha = alpha

    def compute_tversky_index(self, y_true, y_pred):
        true_pos = tf.reduce_sum(y_true * y_pred)
        false_neg = tf.reduce_sum(y_true * (1 - y_pred))
        false_pos = tf.reduce_sum((1 - y_true) * y_pred)

        ti = (true_pos + self.smooth) / (true_pos + self.alpha * false_neg + (1 - self.alpha) * false_pos + self.smooth)
        return ti

    def call(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        score  = self.compute_tversky_index(y_true, y_pred)
        return tf.pow(1-score, self.gamma)
        


In [54]:
# TEST_CASE
from unet import get_unet_128
import numpy as np
import tensorflow.keras.backend as K
model = get_unet_128(num_classes = 3)
x = np.random.uniform(0,1, (10,128,128,3))
target = np.random.randint(0,2, (10,128,128,3))
prediction = model(x, training = True)


l = CustomFocalTverskyLoss()
print(l(target, prediction))
print()
print(focal_tversky(target, prediction))

# # l = tf.keras.losses.CategoricalCrossentropy()
# print(dice_loss(target.astype(np.float32)[...,0][..., np.newaxis], prediction[...,0][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,1][..., np.newaxis], prediction[...,1][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,2][..., np.newaxis], prediction[...,2][..., np.newaxis]))



tf.Tensor(0.70692736, shape=(), dtype=float32)

tf.Tensor(0.70692736, shape=(), dtype=float32)


In [64]:
def log_cosh_dice_loss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, tf.float32)
    x = dice_loss(y_true, y_pred)
    return tf.math.log((tf.exp(x) + tf.exp(-x)) / 2.0)

In [65]:
class CustomLogDiceLoss(tf.keras.losses.Loss):
    def __init__(self, smooth = 1.0 ,name = 'log_dice_loss'):
        super().__init__(name = name)
        self.smooth = smooth

    def compute_dice_coef(self, y_true, y_pred):
        intersection = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
        union = tf.reduce_sum(y_true, [1,2,3]) + tf.reduce_sum(y_pred, [1,2,3])
        score = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return score

    def call(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        score  = self.compute_dice_coef(y_true, y_pred)
        return tf.math.log((tf.exp(1-score) + tf.exp(-(1-score))) / 2.0)

In [66]:
# TEST_CASE
from unet import get_unet_128
import numpy as np
import tensorflow.keras.backend as K
model = get_unet_128(num_classes = 3)
x = np.random.uniform(0,1, (10,128,128,3))
target = np.random.randint(0,2, (10,128,128,3))
prediction = model(x, training = True)


l = CustomLogDiceLoss()
print(l(target, prediction))
print()
print(log_cosh_dice_loss(target, prediction))

# # l = tf.keras.losses.CategoricalCrossentropy()
# print(dice_loss(target.astype(np.float32)[...,0][..., np.newaxis], prediction[...,0][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,1][..., np.newaxis], prediction[...,1][..., np.newaxis]))
# print(dice_loss(target.astype(np.float32)[...,2][..., np.newaxis], prediction[...,2][..., np.newaxis]))



tf.Tensor(0.17049928, shape=(), dtype=float32)

tf.Tensor(0.17050548, shape=(), dtype=float32)
